In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# MNIST Data Pre-processing
import numpy as np
import keras
import tensorflow as tf
from keras.datasets import mnist
from keras.utils import np_utils  # utilities for one-hot encoding of ground truth values

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

target = 1

x_clean = np.load('../AdvGAN/samples/WB-B-t%d-clean.npy' % target)
x_adv = np.load('../AdvGAN/samples/WB-B-t%d-adv.npy' % target)
x_label = np.load('../AdvGAN/samples/WB-B-t%d-label.npy' % target)

%load_ext autoreload
%autoreload 2

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from APEGAN import APEGAN

GAN, G, D = APEGAN([28,28,1])

epochs=100
batch_size=512

np.random.shuffle(x_adv)
np.random.shuffle(x_clean)

N = x_clean.shape[0]
scalarloss = [0,0,0]
start = 0
for cur_epoch in range(epochs):
    idx = range(start, start+batch_size)
    start += batch_size
    if start > N-batch_size:
        start = 0
    x_clean_batch = x_clean[idx,]
    x_adv_batch = x_adv[idx,]
    x_batch = np.concatenate((x_clean_batch, x_adv_batch), axis=0)
    scalarloss[0] = D.train_on_batch(x_batch, np.concatenate((np.ones([batch_size,1]), np.zeros([batch_size,1])), axis=0))
    GAN.train_on_batch(x_adv_batch, [np.ones([batch_size,1]), x_adv_batch])[1:]
    scalarloss[1:] = GAN.train_on_batch(x_adv_batch, [np.ones([batch_size,1]), x_adv_batch])[1:]
    print("Epoch number:",cur_epoch,"; Loss",scalarloss)

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch number: 0 ; Loss [0.27584147, 0.0021352442, 0.31741077]
Epoch number: 1 ; Loss [0.15947926, 0.041754942, 0.2272819]
Epoch number: 2 ; Loss [0.017294228, 0.004940307, 0.2199826]
Epoch number: 3 ; Loss [0.023051597, 0.00039887754, 0.19143327]
Epoch number: 4 ; Loss [0.030081943, 0.0016516674, 0.14611138]
Epoch number: 5 ; Loss [0.015906662, 0.012311277, 0.11609858]
Epoch number: 6 ; Loss [0.007961146, 0.011004613, 0.09629935]
Epoch number: 7 ; Loss [0.0073561096, 0.0024783737, 0.07936717]
Epoch number: 8 ; Loss [0.006720144, 0.0006554317, 0.06779097]
Epoch number: 9 ; Loss [0.007039456, 0.00033149327, 0.062157862]
Epoch number: 10 ; Loss [0.009299325, 0.0016464079, 0.056005817]
Epoch number: 11 ; Loss [0.005610702, 0.0050847, 0.052139126]
Epoch number: 12 ; Loss [0.0050982637, 0.005436663, 0.048606712]
Epoch number: 13 ; Loss [0.005209265, 0.0026805722, 0.04906984]
Epoch number: 14 ; Loss [0.004750479, 0.0023842698, 0.047452066]
Epoch number: 15 ; Loss [0.0038019982, 0.0024695643, 

In [4]:
test_range = x_adv.shape[0]
adv = x_adv[:test_range]
clean = x_clean[:test_range]

print(np.mean(GAN.predict(adv)[0]))
print(np.mean(D.predict(adv)))
print(np.mean(GAN.predict(clean)[0]))
print(np.mean(D.predict(clean)))

0.45165807
0.36795855
0.5094911
0.5141288


In [5]:
import matplotlib.pyplot as plt
import pickle

F = keras.models.load_model('../AdvGAN/models/Classifier-' + 'A' + '.h5')

test_range = x_adv.shape[0]
adv = x_adv[:test_range]
label = x_label[:test_range]
purified = G.predict(adv)
adv_pdt = np.argmax(F.predict(adv), axis=1)
purified_pdt = np.argmax(F.predict(purified), axis=1)

print('adv acc:{:.4f}, rct acc:{:.4f}'.format(np.sum(adv_pdt==label)/test_range, 
                                              np.sum(purified_pdt==label)/test_range))

for k in range(0):
    print(adv_pdt[k])
    plt.imshow((adv[k,] * 255).astype(np.int).reshape(28,28), cmap='gray')
    plt.show()
    print(purified_pdt[k])
    plt.imshow((purified[k,] * 255).astype(np.int).reshape(28,28), cmap='gray')
    plt.show()

adv acc:0.0740, rct acc:0.1040
